In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import pickle
import matplotlib.pyplot as plt
from collections import defaultdict

In [3]:
from mixture_model_base import MixtureModelBase
from mixture_model import MixtureModel
from mixture_model_1S import MixtureModel1S
from order_stat_mixture_model import OrderStatMixtureModel
from order_stat_mixture_model_v2 import OrderStatMixtureModelV2
from xlms import XLMS_Dataset
from myutils import *


00:01:39|TQ-WARNING| Error enabling CUDA. cuda.is_available() returned False. CPU will be used.


cpu


/home/peng.yis/.conda/envs/cuda11.8/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [4]:
def collect_models_for_initskew(initskew_dir):
    p = f'{initskew_dir}/models.pickle'
    if not os.path.exists(p):
        return
    models = pickle.load(open(p, 'rb'))
    # models = pd.DataFrame(models)
    # models = NamedTensor(models.values, names=['randi', 'attr'])
    # print(models.shape)
    # print(
    return models

In [5]:
def collect_models_for_dataset(dataset_dir):
    models = []
    for initskew in os.listdir(dataset_dir):
        d = f'{dataset_dir}/{initskew}'
        if not os.path.isdir(d):
            continue
        x = collect_models_for_initskew(d)
        if x is None:
            continue
        models.append(x)
    return np.array(models)

In [6]:
def collect_models_for_strategy(config, strategy, nsamples=2):
    d = f'../figures_python_diffsign_{nsamples}S2_{config}_{strategy}_initskew_1_2_5/'
    models = []
    for dataset in sorted(os.listdir(d)):
        # if dataset == 'D1810':
        #     continue
        ds_d = d + dataset
        if not os.path.isdir(ds_d):
            continue
        print(dataset)
        x = collect_models_for_dataset(ds_d)
        if x is None:
            return
        models.append(x)
    return np.array(models)

In [7]:
config = 'unweighted_pdf_mode'

In [8]:
models_1s = (collect_models_for_strategy(config, 'gaussian', nsamples=1))
models_1s = np.array(models_1s)

ALott
Alinden
CPSF
D1810
MS2000225
QE
RPA
alban
ecoli_xl
peplib


In [9]:
models_2s = (collect_models_for_strategy(config, 'gaussian'))
models_2s = np.array(models_2s)

ALott
Alinden
CPSF
D1810
MS2000225
QE
RPA
alban
ecoli_xl
peplib


In [10]:
models_2s_nc = (collect_models_for_strategy('no_constraint', 'gaussian'))
models_2s_nc = np.array(models_2s_nc)

ALott
Alinden
CPSF
D1810
MS2000225
QE
RPA
alban
ecoli_xl
peplib


In [13]:
datasets = [
    'ALott',
    'Alinden',
    'CPSF',
    'D1810',
    'MS2000225',
    'QE',
    'RPA',
    'alban',
    'ecoli_xl',
    'peplib',
]

In [14]:
dataset_ind = {d: i for i, d in enumerate(datasets)}

In [12]:
def load_dataset(dataset_name):
    tda_info = json.load(open(f'../results/info/{dataset_name}.json'))
    dataset = XLMS_Dataset(dataset_name, nodup=True)
    # dataset.mat = dataset.mat[:, dataset.mat[1, :] != 0]
    return dataset, tda_info

In [16]:
models_2s.shape

(10, 6, 40)

In [18]:
@gen_ufunc(1, 1)
def get_model_run_time(model):
    return model['timeused']

In [20]:
all_runtime = get_model_run_time(models_2s)
all_runtime.shape

(10, 6, 40)

In [25]:
dataset_runtime = reduce(np.add, all_runtime, (1, 2))

In [26]:
dataset_runtime

array([96164699.47353402, 119428934.154095, 87641573.64798701,
       55143437.81074398, 59190867.655340984, 61943222.628990985,
       50466130.692726, 82402238.46479197, 158423041.93999004,
       58413215.790357016], dtype=object)

In [34]:
@gen_ufunc(1, 1)
def ms_to_hms(ms):
    h = ms / 1000 / 60 // 60
    m = (ms / 1000 - h * 60 * 60) // 60
    s = ms / 1000 - h * 60 * 60 - m * 60
    return f'{h:.0f} hours {m:.0f} minutes {s:.2f} seconds'

In [35]:
ms_to_hms(dataset_runtime)

array(['26 hours 42 minutes 44.70 seconds',
       '33 hours 10 minutes 28.93 seconds',
       '24 hours 20 minutes 41.57 seconds',
       '15 hours 19 minutes 3.44 seconds',
       '16 hours 26 minutes 30.87 seconds',
       '17 hours 12 minutes 23.22 seconds',
       '14 hours 1 minutes 6.13 seconds',
       '22 hours 53 minutes 22.24 seconds',
       '44 hours 0 minutes 23.04 seconds',
       '16 hours 13 minutes 33.22 seconds'], dtype=object)

In [38]:
res = {k: {'runtime': v} for k, v in zip(datasets, ms_to_hms(dataset_runtime))}
df = pd.DataFrame(res).T

In [39]:
df

,runtime
ALott,26 hours 42 minutes 44.70 seconds
Alinden,33 hours 10 minutes 28.93 seconds
CPSF,24 hours 20 minutes 41.57 seconds
D1810,15 hours 19 minutes 3.44 seconds
MS2000225,16 hours 26 minutes 30.87 seconds
QE,17 hours 12 minutes 23.22 seconds
RPA,14 hours 1 minutes 6.13 seconds
alban,22 hours 53 minutes 22.24 seconds
ecoli_xl,44 hours 0 minutes 23.04 seconds
peplib,16 hours 13 minutes 33.22 seconds
